Importação de módulos utilziados no projeto

In [ ]:
pip install scikit-learn


In [4]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
import sys 
import os 

# Adicionando o diretório do projeto ao sys.path
project_path = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
sys.path.append(project_path)


In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA

In [4]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from scipy.sparse import vstack

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [16]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_score

Pré-processamento de dados

Limpeza dos textos 

-- remoção de pontuação, números, caracteres especiais

-- conversão para lower case

In [19]:
import re

# Função de limpeza de texto
def preprocess_text(text):
    # Remover pontuação, números e símbolos especiais
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Converter para minúsculas
    text = text.lower()
    return text

In [22]:
df_original = pd.read_csv('../data/fake_and_real_news.csv')

df_clean = pd.DataFrame({'Text': [], 'label': []})

df_clean['Text'] = df_original['Text'].apply(preprocess_text)

mapeamento = {'Real': 1, 'Fake': 0}
   
# cópia e escrita em disco da coluna 'label' normalizada para o dataframe que contém os dados limpos 
df_clean['label'] = df_original['label'].map(mapeamento)

df_clean.to_csv('../cleaned_data/cleaned_texts.csv', index=False)


In [ ]:
df_clean = pd.read_csv('../cleaned_data/cleaned_texts.csv')

print(df_clean.head())

In [ ]:


# Inicializando o TfidfVectorizer com 1000 features
# pois o conjunto todo é muito grande e resulta em sobrecarga de memória 
vectorizer = TfidfVectorizer(max_features=1000)

# Processamento em partes menores (Chunking) para evitar sobrecarregar a memória.
chunk_size = 1000 
chunks = [df_clean['Text'][i:i + chunk_size] 
          for i in range(0, df_clean.shape[0], chunk_size)]

# Cada parte é transformada usando o TfidfVectorizer.
tfidf_chunks = [vectorizer.fit_transform(chunk) for chunk in chunks]
x = vstack(tfidf_chunks)

# Utilizando o MiniBatchKMeans, que é uma variação de K-means 
# projetada para trabalhar com grandes conjuntos de dados. 
# Ele processa os dados em mini-batches, reduzindo o uso de memória.
num_clusters = 10
kmeans = MiniBatchKMeans(n_clusters= num_clusters, 
                         random_state=0, batch_size=chunk_size).fit(x)

# # Adicionando os rótulos dos clusters ao DataFrame original
df_clean['Cluster'] = kmeans.labels_


# Visualizando os clusters com PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(x.toarray())

plt.scatter(principal_components[:, 0], principal_components[:, 1], 
            c=kmeans.labels_, cmap='rainbow')
plt.title('Clusters de Notícias')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
# plt.savefig('../images/clusterizacao_num10.png', dpi=300)
plt.show()

Classificação

In [46]:

# Carregar os dados
df_clean = pd.read_csv('../cleaned_data/cleaned_texts.csv')

# Separar as features e labels
X = df_clean['Text'].copy()
y = df_clean['label'].copy()

# Dividir os dados em conjuntos de treinamento e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Converter o texto para vetores TF-IDF
vectorizer = TfidfVectorizer()
x_treino_tfidf = vectorizer.fit_transform(x_treino)
x_teste_tfidf = vectorizer.transform(x_teste)

# Verificar os rótulos presentes em y_teste
rotulos_unicos = y_teste.unique()
print(f"Rótulos únicos em y_teste: {rotulos_unicos}")


Rótulos únicos em y_teste: [0 1]


In [ ]:
# Treinar o modelo Decision Tree
modelo_dt = DecisionTreeClassifier(max_depth=10, min_samples_split=50, min_samples_leaf=10)
modelo_dt.fit(x_treino_tfidf, y_treino)

# Obter as características do TfidfVectorizer
nomes_caracteristicas = vectorizer.get_feature_names_out()

# Avaliar o modelo Decision Tree
y_teste_pred_dt = modelo_dt.predict(x_teste_tfidf)
acuracia_dt = accuracy_score(y_teste, y_teste_pred_dt)
precisao_dt = precision_score(y_teste, y_teste_pred_dt)
recall_dt = recall_score(y_teste, y_teste_pred_dt)
f1_dt = f1_score(y_teste, y_teste_pred_dt)

print(f"Acurácia Decision Tree: {acuracia_dt}")
print(f"Precisão Decision Tree: {precisao_dt}")
print(f"Recall Decision Tree: {recall_dt}")
print(f"F1 Score Decision Tree: {f1_dt}")

# Matriz de Confusão Decision Tree
cm_dt = confusion_matrix(y_teste, y_teste_pred_dt)
disp_dt = ConfusionMatrixDisplay(confusion_matrix=cm_dt, display_labels=rotulos_unicos)
disp_dt.plot(cmap='tab20b')
plt.title('Matriz de Confusão para Decision Tree')
# plt.savefig('../images/matriz_confusao_decision_tree.png', dpi=300)
plt.show()


In [ ]:
# Plotar a árvore de decisão
plt.figure(figsize=(20, 10))
plot_tree(modelo_dt, feature_names=nomes_caracteristicas, 
          class_names=['Real', 'Fake'], filled=True)
plt.title('Árvore de Decisão com Palavras')
plt.savefig('../images/arvore_decisao_com_palavras.png', dpi=300)
plt.show()

In [ ]:
# Definir o modelo
mod_nb = DecisionTreeClassifier()

# Executar a validação cruzada
cv_scores_nb = cross_val_score(mod_nb, x_treino_tfidf, y_treino, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores (Árvore de Decisão): {cv_scores_nb}")
print(f"Mean CV Accuracy (Árvore de Decisão): {cv_scores_nb.mean()}")

In [ ]:
# Treinar o modelo Logistic Regression
modelo_lr = LogisticRegression(max_iter=1000)
modelo_lr.fit(x_treino_tfidf, y_treino)

# Avaliar o modelo Logistic Regression
y_teste_pred_lr = modelo_lr.predict(x_teste_tfidf)
acuracia_lr = accuracy_score(y_teste, y_teste_pred_lr)
precisao_lr = precision_score(y_teste, y_teste_pred_lr)
recall_lr = recall_score(y_teste, y_teste_pred_lr)
f1_lr = f1_score(y_teste, y_teste_pred_lr)

print(f"Acurácia Logistic Regression: {acuracia_lr}")
print(f"Precisão Logistic Regression: {precisao_lr}")
print(f"Recall Logistic Regression: {recall_lr}")
print(f"F1 Score Logistic Regression: {f1_lr}")

# Matriz de Confusão Logistic Regression
cm_lr = confusion_matrix(y_teste, y_teste_pred_lr)
disp_lr = ConfusionMatrixDisplay(confusion_matrix=cm_lr, display_labels=rotulos_unicos)
disp_lr.plot(cmap='tab20b')
plt.title('Matriz de Confusão para Logistic Regression')
plt.savefig('../images/matriz_confusao_logistic_regression.png', dpi=300)
plt.show()


In [ ]:
# Definir o modelo
mod_lr = LogisticRegression()

# Executar a validação cruzada
cv_scores_lr = cross_val_score(mod_lr, x_train_tfidf, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores (Logistic Regression): {cv_scores_lr}")
print(f"Mean CV Accuracy (Logistic Regression): {cv_scores_lr.mean()}")

In [ ]:
# Treinar o modelo Naive Bayes
modelo_nb = MultinomialNB()
modelo_nb.fit(x_treino_tfidf, y_treino)

# Avaliar o modelo Naive Bayes
y_teste_pred_nb = modelo_nb.predict(x_teste_tfidf)
acuracia_nb = accuracy_score(y_teste, y_teste_pred_nb)
precisao_nb = precision_score(y_teste, y_teste_pred_nb)
recall_nb = recall_score(y_teste, y_teste_pred_nb)
f1_nb = f1_score(y_teste, y_teste_pred_nb)

print(f"Acurácia Naive Bayes: {acuracia_nb}")
print(f"Precisão Naive Bayes: {precisao_nb}")
print(f"Recall Naive Bayes: {recall_nb}")
print(f"F1 Score Naive Bayes: {f1_nb}")

# Matriz de Confusão Naive Bayes
cm_nb = confusion_matrix(y_teste, y_teste_pred_nb)
disp_nb = ConfusionMatrixDisplay(confusion_matrix=cm_nb, display_labels=rotulos_unicos)
disp_nb.plot(cmap='tab20b')
plt.title('Matriz de Confusão para Naive Bayes')
plt.savefig('../images/matriz_confusao_naive_bayes.png', dpi=300)
plt.show()


In [37]:
# Definir o modelo
mod_nb = MultinomialNB()

# Executar a validação cruzada
cv_scores_nb = cross_val_score(mod_nb, x_train_tfidf, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores (Naive Bayes): {cv_scores_nb}")
print(f"Mean CV Accuracy (Naive Bayes): {cv_scores_nb.mean()}")


Cross-Validation Accuracy Scores (Naive Bayes): [0.97222222 0.95833333 0.96906566 0.96906566 0.96906566]
Mean CV Accuracy (Naive Bayes): 0.967550505050505


In [ ]:
# Redução de Dimensionalidade com TruncatedSVD
svd = TruncatedSVD(n_components=200)  # Ajustar o número de componentes conforme necessário
x_treino_reduzido = svd.fit_transform(x_treino_tfidf)
x_teste_reduzido = svd.transform(x_teste_tfidf)

# Modelo 4: Linear Discriminant Analysis com Redução de Dimensionalidade
# Treinar o modelo Linear Discriminant Analysis
modelo_lda = LinearDiscriminantAnalysis()
modelo_lda.fit(x_treino_reduzido, y_treino)

# Avaliar o modelo Linear Discriminant Analysis
y_teste_pred_lda = modelo_lda.predict(x_teste_reduzido)
acuracia_lda = accuracy_score(y_teste, y_teste_pred_lda)
precisao_lda = precision_score(y_teste, y_teste_pred_lda)
recall_lda = recall_score(y_teste, y_teste_pred_lda)
f1_lda = f1_score(y_teste, y_teste_pred_lda)

print(f"Acurácia Linear Discriminant Analysis: {acuracia_lda}")
print(f"Precisão Linear Discriminant Analysis: {precisao_lda}")
print(f"Recall Linear Discriminant Analysis: {recall_lda}")
print(f"F1 Score Linear Discriminant Analysis: {f1_lda}")

# Matriz de Confusão Linear Discriminant Analysis
cm_lda = confusion_matrix(y_teste, y_teste_pred_lda)
disp_lda = ConfusionMatrixDisplay(confusion_matrix=cm_lda, display_labels=rotulos_unicos)
disp_lda.plot(cmap='tab20b')
plt.title('Matriz de Confusão para Linear Discriminant Analysis')
# plt.savefig('../images/matriz_confusao_lda.png', dpi=300)
plt.show()

In [40]:
# Definir o modelo
mod_lda = LinearDiscriminantAnalysis()

# Executar a validação cruzada
cv_scores_lda = cross_val_score(mod_lda, x_train_tfidf.toarray(), y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores (LDA): {cv_scores_lda}")
print(f"Mean CV Accuracy (LDA): {cv_scores_lda.mean()}")

Cross-Validation Accuracy Scores (LDA): [0.99873737 0.99810606 0.99621212 0.99684343 0.99936869]
Mean CV Accuracy (LDA): 0.9978535353535353


In [ ]:
# Treinar o modelo K-Nearest Neighbors
modelo_knn = KNeighborsClassifier(n_neighbors=5)
modelo_knn.fit(x_treino_tfidf.toarray(), y_treino)  # KNN precisa de matrizes densas

# Avaliar o modelo K-Nearest Neighbors
y_teste_pred_knn = modelo_knn.predict(x_teste_tfidf.toarray())
acuracia_knn = accuracy_score(y_teste, y_teste_pred_knn)
precisao_knn = precision_score(y_teste, y_teste_pred_knn)
recall_knn = recall_score(y_teste, y_teste_pred_knn)
f1_knn = f1_score(y_teste, y_teste_pred_knn)

print(f"Acurácia K-Nearest Neighbors: {acuracia_knn}")
print(f"Precisão K-Nearest Neighbors: {precisao_knn}")
print(f"Recall K-Nearest Neighbors: {recall_knn}")
print(f"F1 Score K-Nearest Neighbors: {f1_knn}")

# Matriz de Confusão K-Nearest Neighbors
cm_knn = confusion_matrix(y_teste, y_teste_pred_knn)
disp_knn = ConfusionMatrixDisplay(confusion_matrix=cm_knn, display_labels=rotulos_unicos)
disp_knn.plot(cmap='tab20b')
plt.title('Matriz de Confusão para K-Nearest Neighbors')
# plt.savefig('../images/matriz_confusao_knn.png', dpi=300)
plt.show()


In [42]:
# Definir o modelo
mod_knn = KNeighborsClassifier()

# Executar a validação cruzada
cv_scores_knn = cross_val_score(mod_knn, x_train_tfidf, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores (KNN): {cv_scores_knn}")
print(f"Mean CV Accuracy (KNN): {cv_scores_knn.mean()}")


Cross-Validation Accuracy Scores (KNN): [0.92739899 0.91035354 0.91224747 0.91540404 0.92424242]
Mean CV Accuracy (KNN): 0.9179292929292929
